In [1]:
# !pip install ctransformers[cuda]==0.2.27 \
#     gradio \
#     chromadb \
#     langchain \
#     torch \
#     sentence-transformers

  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     ---------------------------------------- 50.7/50.7 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     -------------------------------------- 103.4/103.4 kB 6.2 MB/s eta 0:00:00
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4

In [2]:
# !pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.61.tar.gz (37.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.61-cp312-cp312-win_amd64.whl size=2682418 sha256=0020f183f5542bffa0bc80b8afca947a04e294e43aa82cdd2488a0357565c87a
  Stored in directory: c:\users\xayess\appdata\local\pip\cache\wheels\4f\33\94\512ad8dad3943badcb5b4aa048ac827f1b71c53a84b7a948f9
Successfully built llama-cpp-python


In [1]:
import gradio as gr
import time
from ctransformers import AutoModelForCausalLM

c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


def load_llm():
    llm = AutoModelForCausalLM.from_pretrained(
        # "../models/llama-2-13b-chat.Q8_0.gguf",
        "../models/llama-2-7b-chat.Q8_0.gguf",
        model_type='llama',
        max_new_tokens = 2048,
        repetition_penalty = 1.0,
        temperature = 0.75,
        gpu_layers=1000,
        context_length=4096,
    )
    return llm

In [3]:
llm = load_llm()

In [4]:

system_prompt = "You are a travel assistant, your task is to give clear and concise help in creating a travel plan. You have to help the user make a travel plan."
base_prompt = f"<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n[/INST]\n"
end_prompt = "</s>"

def llm_function(message, chat_history):
    print(message)
    print(chat_history)
    full_message = base_prompt
    for m in chat_history:
        full_message += "[INST]" +  m[0] + "[/INST]\n"
        full_message +=  m[1] + "\n"
    full_message += "[INST]" + message + "[/INST]"
    full_message += end_prompt
    response = llm(
        full_message
    )
    print(full_message)
    output_texts = response
    return output_texts

In [5]:
title = "Llama2 7B Quantized GGUF"
"You are a travel assistant, your task is to give clear and concise help in creating a travel plan. You have to help the user make a travel plan, after you may have to scrape for hotels only if the user asks you, whenever you need the hotel informtion output {find_hotels} and then the data in the following format: format: city: , from_date: , to_date: , people: , min_price: , max_price: ."
examples = [
    'Plan a 4 day trip to Japan, include the travel times.',
    'Plan a 5 day trip to Mumbai'
]

In [6]:
gr.ChatInterface(
    fn = llm_function,
    title=title,
    examples=examples
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


I want to travel to Japan for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling alone.
[]
<s>[INST]
<<SYS>>
You are a travel assistant, your task is to give clear and concise help in creating a travel plan. You have to help the user make a travel plan.
<</SYS>>
[/INST]
[INST]I want to travel to Japan for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling alone.[/INST]</s>
Output just the cities to stay overnight in the following format:{ city: , from_date: , to_date: , people: , min_price: , max_price: }.
[['I want to travel to Japan for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling alone.', " Great! Based on your preferences, here's a suggested travel plan for your 8-day trip to Japan:\nDay 1 (June 15): Arrival in Tokyo\n* Arrive at Narita or Haneda airport and take a train or bus to your hotel in central Tokyo\n* Check-in at your hotel and freshen up\n* Spend the af

# RAG Pipeline

In [1]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../data/locations.txt", encoding='UTF-8')

data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n",], chunk_size=1000, chunk_overlap=100)
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)

all_splits = text_splitter.split_documents(data)

Created a chunk of size 1250, which is longer than the specified 1000
Created a chunk of size 1298, which is longer than the specified 1000
Created a chunk of size 1055, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1347, which is longer than the specified 1000
Created a chunk of size 1310, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of size 1331, which is longer than the specified 1000
Created a chunk of size 1363, which is longer than the specified 1000
Created a chunk of size 1359, which is longer than the specified 1000
Created a chunk of size 1032, which is longer than the specified 1000
Created a chunk of size 1435, which is longer than the specified 1000
Created a chunk of size 1157, which is longer than the specified 1000
Created a chunk of size 1271, which is longer than the specified 1000
Created a chunk of s

Created a chunk of size 1288, which is longer than the specified 1000
Created a chunk of size 1057, which is longer than the specified 1000
Created a chunk of size 1731, which is longer than the specified 1000
Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 1304, which is longer than the specified 1000
Created a chunk of size 1177, which is longer than the specified 1000
Created a chunk of size 2051, which is longer than the specified 1000
Created a chunk of size 1138, which is longer than the specified 1000
Created a chunk of size 1304, which is longer than the specified 1000
Created a chunk of size 1061, which is longer than the specified 1000
Created a chunk of size 1031, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1031, which is longer than the specified 1000
Created a chunk of size 1002, which is longer than the specified 1000
Created a chunk of s

In [3]:
print(all_splits)

[Document(page_content='teamLab Planets TOKYO. location_id: 14951238. name: teamLab Planets TOKYO. description: **Exhibition period extended until the end of 2027** teamLab Planets (Toyosu, Tokyo) is a museum where you walk through water, and a garden where you become one with the flowers. It comprises 4 large-scale artwork spaces and 2 gardens created by art collective teamLab. ・Listed by Lonely Planet as one of the "The 7 best museums in Tokyo (that even Tokyoites are waiting in line to visit)". ・Featured by Condé Nast Traveler as one of "The 26 Best Things to Do in Tokyo". ・Winner of "Asia\'s Leading Tourist Attraction 2023" at the World Travel Awards, often referred to as the Oscars of the travel industry.. Address: 6-1-16, Toyosu, Koto 135-0061 Tokyo Prefecture, Japan. latitude: 35.649307. longitude: 139.78976. email: info@planets.art. website: http://www.teamlab.art/e/planets/. rating: 4.5, Category: attraction, Sub category: museums. Trip types: business, couples, solo, family, 

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)

In [4]:
question = "What are places to visit in Kyoto?"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='Street Kart Kyoto. location_id: 14927878. name: Street Kart Kyoto. description: Take On A Street Go-Kart Thru Kyoto! Extremely exciting and a must have experience when you visit Kyoto Japan. Just imagine yourself on a custom made go kart specifically tailored to realize the Real Life SuperHero Go-Karting experience! Dress up in your favorite character costume, playing the music of your choice and driving through the city of Kyoto!!. Address: 23 Kamigoryocho, Higashi Kujo, Minami-Ku, Kyoto 601-8015 Kyoto Prefecture, Japan. latitude: 34.98043. longitude: 135.76224. email: maricarosaka@gmail.com. website: https://kart.st/jp/kyoto.html. rating: 5.0, Category: attraction, Sub category: sightseeing_tours. Trip types: business, couples, solo, family, friends.', metadata={'source': '../data/locations.txt'}),
 Document(page_content='Kyoto Sento Imperial Palace (Sento Gosho). location_id: 1386077. name: Kyoto Sento Imperial Palace (Sento Gosho). Address: Kyotogyoen, Kamig

## Up till that point execute cells


In [13]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
n_gpu_layers = 32 
n_batch = 512
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [13]:
llm = LlamaCpp(
    model_path="../models/llama-2-7b-chat.Q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
    max_new_tokens=2048
)

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: ../models/llama-2-7b-chat.Q8_0.gguf. Received error fileno (type=value_error)

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.llms import HuggingFaceEndpoint

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
llm = HuggingFaceEndpoint(repo_id="google/gemma-7b-it", tokenizer=tokenizer)
# llm = AutoModelForCausalLM.from_pretrained(
#     "google/gemma-7b-it",
#     torch_dtype=torch.bfloat16,
#     max_length=1024,
# )

WARNING! tokenizer is not default parameter.
                    tokenizer was transferred to model_kwargs.
                    Please make sure that tokenizer is what you intended.


ValidationError: 1 validation error for HuggingFaceEndpoint
__root__
  Could not authenticate with huggingface_hub. Please check your API token. (type=value_error)

In [42]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [ ]:
rag_pipeline("Give me a travel itinerary for a 4 day trip to Mumbai.")

/home/varadh/CS_5170/New_ZaWarudoWanderers/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Sure! Based on the information provided, here's a suggested 4-day travel itinerary for Mumbai:
Day 1: Nature and Wildlife
* Start your day at Sanjay Gandhi National Park in the northern suburbs of Mumbai. Spend half a day exploring the park's diverse flora and fauna, including leopards, elephants, and over 200 species of birds.
* Post lunch, head to Chhatrapati Shivaji Maharaj Vastu Sangrahalaya (Formerly the Prince of Wales Museum) in Fort to learn about Mumbai's rich cultural and historical heritage.
Day 2: Scenic Views and Local Cuisine
* Begin your day at Marine Drive and Chowpatty Beach, enjoying the scenic views of the Arabian Sea and taking a stroll along the promenade.
* For lunch, try some local street food at Chowpatty Beach or nearby Juhu Beach.
* In the evening, head to Elephanta Caves on Elephanta Island, accessible by ferry from Gateway of India. Explore

{'query': 'Give me a travel itinerary for a 4 day trip to Mumbai.',
 'result': " Sure! Based on the information provided, here's a suggested 4-day travel itinerary for Mumbai:\nDay 1: Nature and Wildlife\n* Start your day at Sanjay Gandhi National Park in the northern suburbs of Mumbai. Spend half a day exploring the park's diverse flora and fauna, including leopards, elephants, and over 200 species of birds.\n* Post lunch, head to Chhatrapati Shivaji Maharaj Vastu Sangrahalaya (Formerly the Prince of Wales Museum) in Fort to learn about Mumbai's rich cultural and historical heritage.\nDay 2: Scenic Views and Local Cuisine\n* Begin your day at Marine Drive and Chowpatty Beach, enjoying the scenic views of the Arabian Sea and taking a stroll along the promenade.\n* For lunch, try some local street food at Chowpatty Beach or nearby Juhu Beach.\n* In the evening, head to Elephanta Caves on Elephanta Island, accessible by ferry from Gateway of India. Explore"}

In [7]:
gr.ChatInterface(
    fn = llm_function,
    title=title,
    examples=examples
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Hello, who are you?
[]
<s>[INST]
<<SYS>>
You are a travel assistant, your task is to give clear and concise help in creating a travel plan. You have the ability to scrape for hotels, whenever you feel the need to output {find_hotels} and the data in the following format: format: city: , from_date: , to_date: , people: , min_price: , max_price: .
<</SYS>>
[/INST]
[INST]Hello, who are you?[/INST]</s>
I want to travel to Japan from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling alone.
[['Hello, who are you?', "Hello! I'm your personal travel assistant, here to help you plan your next adventure. I can help you find the best hotels, flights, and activities for your trip. What city are you interested in traveling to? Can you please provide me with the following information:\n* City:\n* From date:\n* To date:\n* Number of people traveling:\n* Minimum price range:\n* Maximum price range:\n\nOnce I have this information, I can start scraping for hotels that fit you

# LLAMA 2 WITH RAG NOT CPP

In [4]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [5]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [6]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    #quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.40s/it]


Prepare model, tokenizer: 23.465 sec.


In [7]:
# def test_model(tokenizer, pipeline, prompt_to_test):
#     """
#     Perform a query
#     print the result
#     Args:
#         tokenizer: the tokenizer
#         pipeline: the pipeline
#         prompt_to_test: the prompt
#     Returns
#         None
#     """
#     # adapted from https://huggingface.co/blog/llama2#using-transformers
#     time_1 = time()
#     sequences = pipeline(
#         prompt_to_test,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         max_length=200,)
#     time_2 = time()
#     print(f"Test inference: {round(time_2-time_1, 3)} sec.")
#     for seq in sequences:
#         print(f"Result: {seq['generated_text']}")

In [8]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 0.092 sec.


In [11]:
# test_model(tokenizer,
#            query_pipeline,
#            "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

NameError: name 'test_model' is not defined

In [9]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [14]:
# llm("[INST]Give me a travel itinerary for a trip to Japan from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars. Come up with interesting locations where the user has to stay. Output just the cities to stay in the following format: city: , from_date: , to_date: , people: , min_price: , max_price: .[/INST]")

"[INST]Give me a travel itinerary for a trip to Japan from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars. Come up with interesting locations where the user has to stay. Output just the cities to stay in the following format: city: , from_date: , to_date: , people: , min_price: , max_price: .[/INST]  Sure, here's a 7-day travel itinerary for a trip to Japan from June 15th to June 22nd with a budget of $5000:\n\nCity: Tokyo,, from_date: 2024-06-15, to_date: 2024-06-19, people: 1, min_price: $100, max_price: $200\n\nCity: Kyoto,, from_date: 2024-06-19, to_date: 2024-06-22, people: 1, min_price: $80, max_price: $150\n\nHere's a brief description of each location and why it was chosen:\n\nTokyo: Tokyo is Japan's capital and largest city, with a rich history, vibrant culture, and cutting-edge technology. It's a must-visit destination for any traveler to Japan, with iconic landmarks like the Tokyo Tower, the Imperial Palace, and the Tsukiji Fish Market. Staying in Tokyo for 4 nights 

In [11]:
# query1 = []
# query1.append({"role":"user","content":"[INST]My name is Joj[/INST]"})
# resp1 = query_pipeline(query1) #llm(query1)
# print(resp1)
# q2 = resp1
# q2.append({"role":"user","content":"[INST]What is my name?[/INST]"})
# print(q2)
# resp2 = query_pipeline(q2) #llm(query1)
# print(resp2)


[{'generated_text': [{'role': 'user', 'content': '[INST]My name is Joj[/INST]'}, {'role': 'assistant', 'content': "  Hello Joj! It's nice to meet you. Is there something I can help you with or would you like to chat?"}]}]
[{'generated_text': [{'role': 'user', 'content': '[INST]My name is Joj[/INST]'}, {'role': 'assistant', 'content': "  Hello Joj! It's nice to meet you. Is there something I can help you with or would you like to chat?"}]}, {'role': 'user', 'content': '[INST]What is my name?[/INST]'}]


ValueError: When passing chat dicts as input, each dict must have a 'role' and 'content' key.

In [ ]:
# "Output just the cities to stay overnight in the exactly following format: city: {city} , from_date: {yyyy-mm-dd} , to_date: {yyyy-mm-dd}, people: {people}, min_price: {min_price}, max_price: {max_price}."
# llm("[INST]Give me a travel itinerary for a trip to Japan from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars. Come up with interesting locations where the user has to stay. ")

In [10]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [12]:
# rag_pipeline("Give me a travel itinerary for a trip to Japan from 2024-06-15 to 2024-06-22 with a budget of 4000 dollars. Come up with interesting locations, you may use the provided context to help answer the question. Output just the cities to stay in the following format: city: , from_date: , to_date: , people: , min_price: , max_price: .")

c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Give me a travel itinerary for a 3 day trip to Japan from 2024-06-15 to 2024-06-22 with a budget of 4000 dollars. Come up with interesting locations, you may use the provided context to help answer the question. Output just the cities to stay in the following format: city: , from_date: , to_date: , people: , min_price: , max_price: .',
 'result': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nJapan Panoramic Tours. location_id: 9741667. name: Japan Panoramic Tours. description: Hello everyone! We are Japan Panoramic Tours. We have run "one day bus trip" for about 7 years since 2016 and had about 70,000 passengers from over 135 countries so far. Our goal is to get international travelers to become big Japan fans through our tours. Thanks to such wonderful reviews written by past passengers on tripadvisor, we have received "Certificate of Excellence" in fo

In [8]:
# from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
# from langchain_core.messages import SystemMessage, HumanMessage

# human_prompt = HumanMessagePromptTemplate.from_template("[INST]{message}[/INST]")
# system_prompt = SystemMessagePromptTemplate.from_template("[INST]{message}[/INST]")
# base_prompt = f"<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n[/INST]\n"
# chat_template = ChatPromptTemplate.from_messages(
#     [
#         SystemMessage(
#             content=(
#                 "<s>[INST]\n<<SYS>>\nYou are a travel assistant, your task is to give clear and concise help in creating a travel plan\n<</SYS>>\n[/INST]\n"
#             )
#         ),
#         human_prompt,
#     ]
# )


In [12]:
from llm_integration import DataExtractor
from hscraper import find_hotels
import sys
from langchain.chains import LLMChain

parent_dir = '..'
sys.path.append(parent_dir)

def hotel_tool(query):
    hotels = []
    for line in query.split('\n'):
        params = DataExtractor().extract(line)
        if len(params) > 0 and 'City' in params:
            hotels.append(find_hotels(params['City'], params['from_date'], params['to_date'], params['people'], 0, params["min_price"], params['max_price']))
    return hotels

200
https://www.booking.com/searchresults.html?ss=Tokyo&lang=en-us&sb=1&src_elem=sb&src=index&checkin=2024-06-15&checkout=2024-06-17&group_adults=2&no_rooms=1&group_children=0&nflt=price%3DUSD-600-1000-1
ids are  19
ids are  18
ids are  85
ids are  17
ids are  19
[[[{'room': 'Modern Japanese-Style Premium Room - Non Smoking', 'price': 'US$1,226'}, {'room': 'Modern Japanese-Style Premium Room - Non Smoking', 'price': 'US$887'}, {'room': 'Modern Japanese-Style Premium Room - Non Smoking', 'price': 'US$982'}, {'room': 'Modern Japanese-Style Premium Room - Non Smoking', 'price': 'US$986'}, {'room': 'Modern Japanese-Style Premium Room - Non Smoking', 'price': 'US$1,106'}, {'room': 'Modern Japanese-Style Premium Room - Non Smoking', 'price': 'US$1,082'}, {'room': 'Double Decker Room - Non Smoking', 'price': 'US$167'}, {'room': 'Double Decker Room - Non Smoking', 'price': 'US$209'}, {'room': 'Double Decker Room - Non Smoking', 'price': 'US$215'}, {'room': 'Standard Twin Room - Non-Smoking', '

In [27]:
# hotel_tool("Helpful Answer:\ncity: Tokyo, from_date: 2024-06-15, to_date: 2024-06-17, people: 2, min_price: 200, max_price: 400\ncity: Hiroshima, from_date: 2024-06-17, to_date: 2024-06-19, people: 2, min_price: 200, max_price: 400\ncity: Kobe, from_date: 2024-06-19, to_date: 2024-06-22, people: 2, min_price: 200, max_price: 400\n\nNote: The budget of 4000 dollars is assumed to be for 2 people. The min_price and max_price are for each location, and are based on the provided context.")
# hotel_tool(" Sure, here are the cities to stay overnight during the trip to Japan based on the travel plan provided:City: Tokyo, from_date: 2023-03-01, to_date: 2023-03-04, people: 1, min_price: 10000, max_price: 20000.\nCity: Kamakura, from_date: 2023-03-04, to_date: 2023-03-05, people: 1, min_price: 8000, max_price: 15000.")

https://www.booking.com/searchresults.html?ss=Tokyo&lang=en-us&sb=1&src_elem=sb&src=index&checkin=2023-03-01&checkout=2023-03-04&group_adults=1&no_rooms=1&group_children=0&nflt=price%3DUSD-10000-20000.-1
ids are  0
ids are  0
ids are  0
ids are  0
ids are  0
[[[], {'name': 'Hotel Yaenomidori Tokyo', 'address': '104-0032 Tokyo-to, Chuo-ku Hatchobori 2-13-1, Japan', 'rating': '8.6', 'stars': 4}, ['Non-smoking rooms', 'Free WiFi', 'Parking', '2 restaurants', '24-hour front desk', 'Facilities for disabled guests', 'Laundry', 'Lift', 'Bar', 'Good breakfast', 'Non-smoking rooms', 'Free WiFi', 'Parking', '2 restaurants', '24-hour front desk', 'Facilities for disabled guests', 'Laundry', 'Lift', 'Bar', 'Good breakfast', 'Toilet paper', 'Towels', 'Bidet', 'Bath or shower', 'Slippers', 'Private bathroom', 'Toilet', 'Free toiletries', 'Hairdryer', 'Bath', 'Shower', 'Linen', 'Garden', 'Electric kettle', 'Refrigerator', 'Socket near the bed', 'Clothes rack', 'Sofa', 'Desk', 'Flat-screen TV', 'Telep

['Hotel info: name: Hotel Yaenomidori Tokyo, address: 104-0032 Tokyo-to, Chuo-ku Hatchobori 2-13-1, Japan, rating: 8.6, stars: 4, Non-smoking rooms, Free WiFi, Parking, 2 restaurants, 24-hour front desk, Facilities for disabled guests, Laundry, Lift, Bar, Good breakfast, Non-smoking rooms, Free WiFi, Parking, 2 restaurants, 24-hour front desk, Facilities for disabled guests, Laundry, Lift, Bar, Good breakfast, Toilet paper, Towels, Bidet, Bath or shower, Slippers, Private bathroom, Toilet, Free toiletries, Hairdryer, Bath, Shower, Linen, Garden, Electric kettle, Refrigerator, Socket near the bed, Clothes rack, Sofa, Desk, Flat-screen TV, Telephone, TV, Wine/champagne, Bar, Restaurant, Parking garage, Accessible parking, Invoice provided, Luggage storage, 24-hour front desk, Daily housekeeping, Trouser press, Dry cleaning, Laundry, Fax/photocopying, Fire extinguishers, CCTV outside property, CCTV in common areas, Smoke alarms, Security alarm, Key card access, 24-hour security, Safety de

In [17]:
system_prompt = "You are a travel assistant, your task is to give clear and concise help in creating a travel plan. You have to help the user make a travel plan."
base_prompt = f"<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n[/INST]\n"
end_prompt = "</s>"
formatter_prompt = "You are a formetter, your task is to format the data exactly by the given template. Output just the cities to stay overnight from the travel plan in the exactly this format: city: {city}, from_date: {yyyy-mm-dd}, to_date: {yyyy-mm-dd}, people: {people}, min_price: {min_price}, max_price: {max_price}.[/INST]"


def llm_cycle(query, chat_history):
    full_message = base_prompt
    for m in chat_history:
        full_message += "[INST]" +  m[0] + "[/INST]\n"
        full_message +=  m[1] + "\n"
    
    print(full_message)
    if "hotel" in query.lower():
        format_query = "<s>[INST]\n<<SYS>>\n You are a formetter, your task is to format the data exactly by the given template. Output just the cities to stay overnight from the travel plan in the exactly this format: city: {city}, from_date: {yyyy-mm-dd}, to_date: {yyyy-mm-dd}, people: {people}, min_price: {min_price}, max_price: {max_price}.[/INST]\n<</SYS>>\n[/INST]\n" + chat_history[-1][1] if len(chat_history) > 0 else query + "[INST] Output just the cities to stay overnight from the travel plan in the exactly this format: City: {city}, from_date: {yyyy-mm-dd}, to_date: {yyyy-mm-dd}, people: {people}, min_price: {min_price}, max_price: {max_price}.[/INST]</s>"
        print("Format query: ", format_query)
        formatted = llm(format_query)
        formatted = formatted[formatted.find("[/INST]"):]
        print("Formatted: ", formatted)
        hotels = hotel_tool(formatted)
        hotels_str = "\n ".join(hotels)
        query = "The following hotels are available, propose some to the user: " + hotels_str + ". " + query

    full_message += "[INST]" + query + "[/INST]"
    full_message += end_prompt
    print(full_message)
    if len(full_message) > 4000:
        full_message = full_message[len(full_message) - 2000:]
    result = rag_pipeline(full_message)["result"]
    return result



In [34]:
# from langchain.chains import LLMChain
# from llm_integration import DataExtractor


# # Define a function that coordinates the retrieval and generation process
# def travel_planner(chain, context):
#     # Use the RAG pipeline to get location suggestions based on the context
#     location_suggestions = chain.llm.query_pipeline(context)
    
#     # For simplicity, assume the first suggestion is what we want
#     chosen_location = location_suggestions[0]
    
#     # Now use the HotelTool to get hotels for the chosen location
#     # Assume we parse the location to extract city and dates from `chosen_location`
#     # You will need to adjust parsing based on your actual data format
#     params = DataExtractor.extract(chosen_location)
#     hotels = chain.hotel_tool.call(params['city'], params['from'], params['to'], params['people'], 0, params["min_price"], params['max_price'])
    
#     # Format the final output
#     return f"Visit {chosen_location}. Hotels available: {hotels}"

# # Assuming `query_pipeline` and `rag_pipeline` are set up
# hotel_tool = HotelTool

# # Add the custom function as a prompt in a chain
# chain = LLMChain(llm=llm, prompt = prompt)
# chain.bind_tools(hotel_tool)  # Adding the hotel tool to the chain for access in the function

# # Example usage
# context = "I want to travel to Japan for a cultural experience. city:Tokyo, from:2024-06-15, to:2024-06-29, people:2, min_price:min, max_price:2000"
# response = chain.run(context)
# print(response)

ModuleNotFoundError: No module named 'llm_integration'

In [14]:
title = "Travel Assistant"	
examples = [
    "I want to travel to Japan for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling alone.",
    "I want to travel to Mumbai for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling with a partner."
]

In [18]:
import gradio as gr

gr.ChatInterface(
    fn = llm_cycle,
    title=title,
    examples=examples
).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


<s>[INST]
<<SYS>>
You are a travel assistant, your task is to give clear and concise help in creating a travel plan. You have to help the user make a travel plan.
<</SYS>>
[/INST]

Format query:  Wow, 4 days in Japan sounds like a great trip! With a budget of $5000, you'll have plenty of opportunities to explore the country's vibrant cities, rich culture, and delicious food. Here are some suggestions for your 4-day trip to Japan:

Day 1: Arrival in Tokyo and Exploring the City

Arrive at Narita or Haneda airport and take a train or bus to your hotel
Visit the famous Shibuya Crossing and take a walk around the Shibuya area
Explore the trendy Harajuku district and shop for some unique fashion items
Visit the Meiji Shrine, a beautiful Shinto shrine located in a peaceful forested area
Day 2: Tokyo's Must-See Attractions

Visit the Tokyo Skytree for panoramic views of the city
Explore the historic Asakusa district and visit the famous Senso-ji Temple
Visit the Imperial Palace, the main resi

Wow, 4 days in Japan sounds like a great trip! With a budget of $5000, you'll have plenty of opportunities to explore the country's vibrant cities, rich culture, and delicious food. Here are some suggestions for your 4-day trip to Japan:

Day 1: Arrival in Tokyo and Exploring the City

Arrive at Narita or Haneda airport and take a train or bus to your hotel
Visit the famous Shibuya Crossing and take a walk around the Shibuya area
Explore the trendy Harajuku district and shop for some unique fashion items
Visit the Meiji Shrine, a beautiful Shinto shrine located in a peaceful forested area
Day 2: Tokyo's Must-See Attractions

Visit the Tokyo Skytree for panoramic views of the city
Explore the historic Asakusa district and visit the famous Senso-ji Temple
Visit the Imperial Palace, the main residence of the Emperor of Japan
Take a stroll through the beautiful Hamarikyu Gardens, a former duck hunting ground for the Imperial family
Day 3: Foodie Adventures in Tokyo

Start the day with a visit to the Tsukiji Fish Market, one of the largest fish markets in the world
Take a sushi-making class and learn how to make some of Japan's famous dishes
Visit the trendy Roppongi Hills area and try some of the city's best food and drinks
Explore the colorful and vibrant streets of Shinjuku and enjoy some of the city's nightlife
Day 4: Day Trip to Kamakura

Take a train to Kamakura, a coastal city located about an hour away from Tokyo
Visit the famous Great Buddha of Kamakura, a 13th-century bronze statue
Explore the beautiful temples and shrines of Kamakura, including the Hasedera Temple and the Engaku-ji Temple
Enjoy some fresh seafood at one of the local restaurants
Budget Breakdown:

Transportation: $100 (train tickets, bus fares, etc.)
Accommodation: $150 (hotel costs)
Food and Drink: $300 (meals, snacks, etc.)
Attractions and Activities: $250 (entry fees, etc.)
Total Budget: $500

Of course, this is just a rough estimate, and your actual budget may vary depending on your personal preferences and spending habits. But with $5000, you should have plenty of money to explore Japan and experience all it has to offer!

I hope this helps you plan your trip to Japan! Let me know if you have any other questions.

This is the plan for my trip to Japan, find me the hotels for the trip